<a href="https://colab.research.google.com/github/kesaroid/website-to-pdf/blob/main/RAG_on_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG to read PDF

### Imports

In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 76.2 MB/s eta 0:0

In [1]:
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain.vectorstores.chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

In [2]:
# supress langchain deprecation warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
DATA_PATH = 'data/'
CHROMA_PATH = "chroma"

In [4]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

### Run Ollama server on colab

In [5]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:4 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,303 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [62.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:13 http://security.ubuntu.com/ubuntu jam

In [27]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

In [7]:
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 970aa74c0a90...   0% ▕▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕▏ 239 KB/274 MB                  pulling manifest 
pulling 970aa74c0a90...   4% ▕▏  10 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  18% ▕▏  49 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  31% ▕▏  86 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  37% ▕▏ 100 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  45% ▕▏ 122 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  55% ▕▏ 150 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  60% ▕▏ 164 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90

In [28]:
# pull an ollama model

# !ollama pull llama3.1:8b
!ollama pull deepseek-llm:7b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 60cfdbde0472...   0% ▕▏    0 B/4.0 GB                  pulling manifest 
pulling 60cfdbde0472...   0% ▕▏    0 B/4.0 GB                  pulling manifest 
pulling 60cfdbde0472...   0% ▕▏    0 B/4.0 GB                  pulling manifest 
pulling 60cfdbde0472...   1% ▕▏  28 MB/4.0 GB                  pulling manifest 
pulling 60cfdbde0472...   2% ▕▏  71 MB/4.0 GB                  pulling manifest 
pulling 60cfdbde0472...   4% ▕▏ 172 MB/4.0 GB                  pulling manifest 
pulling 60cfdbde0472...   7% ▕▏ 277 MB/4.0 GB                  pulling manifest 
pulling 60cfdbde0472...   8% ▕▏ 332 MB/4.0 GB                  pulling manifest 
pulling 60cfdbde0472...  11% ▕▏ 436 MB/4.0 GB                  pulling manifest 
pulling 60cfdbde0472...  14% ▕▏ 541 MB/4.0 GB                  pulling manifest 
pulling 60cfdbde0472...  15% ▕▏ 590 MB/

### Load data

In [9]:
def read_docs(DATA_PATH):
  document_loader = PyPDFDirectoryLoader(DATA_PATH)
  documents = document_loader.load()

  text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )

  chunks = text_splitter.split_documents(documents)

  return chunks

In [10]:
from langchain_community.embeddings.ollama import OllamaEmbeddings
# from langchain_community.embeddings.bedrock import BedrockEmbeddings

def embedding_func():
  embeddings = OllamaEmbeddings(model="nomic-embed-text")
  return embeddings

In [11]:
def calculate_chunk_ids(chunks):

    # This will create IDs like "data/monopoly.pdf:6:2"
    # Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id

    return chunks

In [12]:
def add_to_vector_db(chunks):
  db = Chroma(
        persist_directory=CHROMA_PATH, embedding_function=embedding_func()
    )

  # Calculate Page IDs.
  chunks_with_ids = calculate_chunk_ids(chunks)

  # Add or Update the documents.
  existing_items = db.get(include=[])  # IDs are always included by default
  existing_ids = set(existing_items["ids"])
  print(f"Number of existing documents in DB: {len(existing_ids)}")

  # Only add documents that don't exist in the DB.
  new_chunks = []
  for chunk in chunks_with_ids:
      if chunk.metadata["id"] not in existing_ids:
          new_chunks.append(chunk)

  if len(new_chunks):
      print(f"Adding new documents: {len(new_chunks)}")
      new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
      db.add_documents(new_chunks, ids=new_chunk_ids)
      db.persist()
  else:
      print("No new documents to add")

  return db

In [13]:
chunks = read_docs(DATA_PATH)

In [14]:
chunks[-1]

Document(metadata={'source': 'data/hipandkneebook.pdf', 'page': 644, 'page_label': '645'}, page_content='MANASQUAN, NJ, USA\nPARCELLS@GMAIL.COM\nprivacy policy\n; \nterms & conditions\n; \nsitemap\n; \nauthor Bert Parcells\nCOVER\nKNEE\tCHAPTERS\nHIP\tCHAPTERS\nPERI-OP\tCARE\nIMPLANTS\nHIP & KNEE BOOK')

In [15]:
db = add_to_vector_db(chunks)

Number of existing documents in DB: 0
Adding new documents: 1424


### Query RAG

In [16]:
def query_db(query_text, db):
  results = db.similarity_search_with_score(query_text, k=5)
  return results

In [35]:
def rag_query(query_text, use_model='deepseek-r1:7b'):
  # query vector db
  results = query_db(query_text, db)

  context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
  prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
  prompt = prompt_template.format(context=context_text, question=query_text)

  model = OllamaLLM(model = use_model)
  response_text = model.invoke(prompt)

  sources = [doc.metadata.get("id", None) for doc, _score in results]
  formatted_response = f"Response: {response_text}\nSources: {sources}"
  print(formatted_response)
  return response_text


In [31]:
result_text = rag_query("What does LFC stand for?")

Response: Based on the provided context, LFC stands for "lateral femoral condyle."
Sources: ['data/hipandkneebook.pdf:10:1', 'data/hipandkneebook.pdf:12:1', 'data/hipandkneebook.pdf:402:2', 'data/hipandkneebook.pdf:13:1', 'data/hipandkneebook.pdf:398:1']


### Test

In [36]:
# 1 CoT model
rag_query("What is the biomechanical center of rotation of a distal femur?")

Response: <think>
Okay, so I need to figure out what the biomechanical center of rotation of a distal femur is based on the given context. Let me read through the provided text carefully.

The first paragraph talks about patellar maltracking and posterior rollback being important for TKA (total knee arthroplasty). It mentions that the point of terminal flexion determines how the knee moves, which is crucial for the TKA's success. There's also a mention of internal rotation causing issues with patellar tracking and the flexion gap.

Moving on, there's a section about determining the femoral implant size using a 4-in-1 cutting guide. It explains that the guide helps in making the necessary cuts based on the femur size. Then it discusses rotation—internal vs external—and how internal rotation affects patellar tracking and the flexion gap. The context warns about severe internal rotation causing dislocation, which is bad for the joint.

Next, there's a part about the distal femur cut affec

"<think>\nOkay, so I need to figure out what the biomechanical center of rotation of a distal femur is based on the given context. Let me read through the provided text carefully.\n\nThe first paragraph talks about patellar maltracking and posterior rollback being important for TKA (total knee arthroplasty). It mentions that the point of terminal flexion determines how the knee moves, which is crucial for the TKA's success. There's also a mention of internal rotation causing issues with patellar tracking and the flexion gap.\n\nMoving on, there's a section about determining the femoral implant size using a 4-in-1 cutting guide. It explains that the guide helps in making the necessary cuts based on the femur size. Then it discusses rotation—internal vs external—and how internal rotation affects patellar tracking and the flexion gap. The context warns about severe internal rotation causing dislocation, which is bad for the joint.\n\nNext, there's a part about the distal femur cut affecti

In [33]:
# 2 Regular LLM output
rag_query("How is the posterior referencing performed? and its uses")

Response: The posterior referencing in total knee arthroplasty (TKA) involves measuring the AP diameter of the femur to ensure that the size of the femoral component aligns with the normal posterior condylar offset. This process helps maximize the distance before impingement occurs, similar to how a larger femoral head would affect hip motion. The use of a posterior referencing system guarantees a normal posterior condylar offset and allows for a greater arc of motion during knee flexion while maintaining the patellofemoral joint's integrity.
Sources: ['data/hipandkneebook.pdf:32:1', 'data/hipandkneebook.pdf:48:1', 'data/hipandkneebook.pdf:48:0', 'data/hipandkneebook.pdf:542:1', 'data/hipandkneebook.pdf:69:1']


"The posterior referencing in total knee arthroplasty (TKA) involves measuring the AP diameter of the femur to ensure that the size of the femoral component aligns with the normal posterior condylar offset. This process helps maximize the distance before impingement occurs, similar to how a larger femoral head would affect hip motion. The use of a posterior referencing system guarantees a normal posterior condylar offset and allows for a greater arc of motion during knee flexion while maintaining the patellofemoral joint's integrity."

In [37]:
# 3 CoT model
rag_query("How to treat a bone lose during tka")

Response: <think>
Okay, so I need to figure out how to treat a loosened bone during a TKA. From the context provided, there's a chapter called "ASEPTIC LOOSENING" which seems relevant. Let me read through that section again.

The section mentions wear patterns and classification systems for evaluating loosening after TKA. It also talks about treatment options and outcomes. So, understanding the wear pattern is important first. Wear can be due to several reasons like overloading, component placement issues, or even manufacturing defects.

I remember from my studies that in cases of bone loosening post-TKA, the treatment often starts with a thorough clinical evaluation. That means checking how the patient is moving and experiencing pain, which helps determine if it's a complication or just wear.

Next, imaging like X-rays might show increased wear areas, indicating where the issue lies. If the wear is significant enough that the implant isn't performing well, surgery might be necessary t

'<think>\nOkay, so I need to figure out how to treat a loosened bone during a TKA. From the context provided, there\'s a chapter called "ASEPTIC LOOSENING" which seems relevant. Let me read through that section again.\n\nThe section mentions wear patterns and classification systems for evaluating loosening after TKA. It also talks about treatment options and outcomes. So, understanding the wear pattern is important first. Wear can be due to several reasons like overloading, component placement issues, or even manufacturing defects.\n\nI remember from my studies that in cases of bone loosening post-TKA, the treatment often starts with a thorough clinical evaluation. That means checking how the patient is moving and experiencing pain, which helps determine if it\'s a complication or just wear.\n\nNext, imaging like X-rays might show increased wear areas, indicating where the issue lies. If the wear is significant enough that the implant isn\'t performing well, surgery might be necessary 